In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import *
import numpy as np
from sklearn.decomposition import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

C:\Users\Hrafnkell\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Hrafnkell\Anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Users\Hrafnkell\Anaconda2\lib\site-packages\sklearn\lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
C:\Users\

In [2]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 60

In [3]:
##Define the object for the season team
class MyDf:
    def __init__(self, mydf):
        self.df = mydf 

In [4]:
def select_season (df, season):
    return df[df['season']==season]

In [5]:
def cut_out_n_m_matches(df,n,m):
    a=df['H_ROUND'].unique()[n-1]
    b=df['H_ROUND'].unique()[-m]
    
    return df[(df['H_ROUND']>a) & (df['H_ROUND']<b)]
    

In [6]:
def train_test_index(df,predicted_season):
    train_index=df[df.season!=predicted_season].index
    test_index=df[df.season==predicted_season].index
    return train_index, test_index

In [7]:
def train_test_index_only_round(df,predicted_round):
    train_index=df[df['H_ROUND']<predicted_round].index
    test_index=df[df['H_ROUND']==predicted_round].index
    return train_index, test_index

In [8]:
def split_train_test_data(df,train_index,test_index,col_train,col_pred,i):
        
    X_train=df.loc[train_index].iloc[:,col_train]
    Y_train=df.loc[train_index][col_pred]

   
    
    X_test=df.loc[test_index]
    Y_test=df.loc[test_index]

    X_test=X_test[X_test.H_ROUND==i].iloc[:,col_train]
    Y_test=Y_test[Y_test.H_ROUND==i][col_pred]

    return X_train, Y_train, X_test, Y_test

In [9]:
def Predict_whole_season_by_one (BIG, predseason, col_train, col_pred, method, method_name):
        
    tmp=pd.DataFrame()
    
    train_index, test_index = train_test_index(BIG,predseason)
    lis=[]
    
    for i in BIG[BIG.season==predseason].H_ROUND.unique():
       
        X_train,Y_train,X_test,Y_test = split_train_test_data(BIG,train_index,test_index,col_train,col_pred,i) 
        method.fit(X_train,Y_train)
        #print(method.score(X_test, Y_test))
        #print('hi')
        lis.append(float(method.score(X_test, Y_test)))

    tmp[method_name]=lis 
    
    return tmp

In [10]:
def Predict_whole_season_with_one (BIG, predseason, col_train, col_pred,method, method_name):
        
    tmp=pd.DataFrame()    
    
    train_index, test_index = train_test_index(BIG,predseason)
    lis=[]

    
    for i in BIG[BIG.season==predseason].H_ROUND.unique():
       
        X_train,Y_train,X_test,Y_test = split_train_test_data(BIG,train_index,test_index,col_train,col_pred,i) 
        method.fit(X_train,Y_train)
        lis.append(float(method.score(X_test, Y_test)))
        
        train_index=list(train_index)+list(X_test.index)
       
    tmp[method_name]=lis     
    return tmp
    

In [11]:
def Predict_season_with_current_season (df, predseason, col_train, col_pred,method, method_name):
    
    tmp=pd.DataFrame()
    tmp_2=df[df['season']==predseason]
    lis=[]
    
    second_round=tmp_2['H_ROUND'].unique()[1]
    
        
    for i in range(int(df.H_ROUND.unique()[1]), int(df.H_ROUND.unique()[-1])):
        
        train_index, test_index = train_test_index_only_round(df,i)
        
        
        
        X_train=df.loc[train_index].iloc[:,col_train]
        Y_train=df.loc[train_index][col_pred]

        
        X_test=df.loc[test_index].iloc[:,col_train]
        Y_test=df.loc[test_index][col_pred]
        
       
        method.fit(X_train,Y_train)
        
        lis.append(float(method.score(X_test, Y_test)))
        
       
    tmp[method_name]=lis     
    
    return tmp


In [12]:
def find_best_model (X,col_pred,col_train,method):
    n=100
    from sklearn.cross_validation import train_test_split
    tmp=0
    for i in range(n):
        X_train, X_test, y_train, y_test = train_test_split(X.iloc[:,column_to_use_as_prediction], X[col_pred], test_size=0.20)
        method.fit(X_train,y_train)
        if method.score(X_test,y_test)>tmp:
            Best=method.fit(X_train,y_train)
    return Best

# Loading data

In [13]:
final=pd.read_csv('../Data/from_sqlite_Data/realfinal.csv')
final.drop('Unnamed: 0',axis=1,inplace=True)
allchampionships=[]

In [14]:
for i in range(len(final.country_id.unique())):
    allchampionships.append(MyDf(final[final['country_id']==final.country_id.unique()[i]].sort(columns='Date')))
    allchampionships[i].df.reset_index(inplace=True)
    allchampionships[i].df.drop(['index'],axis=1,inplace=True)

C:\Users\Hrafnkell\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


# Cut out first and last n,m matches

In [15]:
n=4
m=4

In [16]:
for i in range(len(final.country_id.unique())):
    allchampionships[i].df=cut_out_n_m_matches(allchampionships[i].df,n,m)
    

In [17]:
pd.DataFrame(allchampionships[0].df.columns)

,0
0,country_id
1,season
2,Date
3,HomeTeam
4,AwayTeam
5,FTHG
6,FTAG
7,FTR
8,H_ROUND
9,H_TP3


## SVM_LINEAR

In [98]:
season_to_be_predicted='2015/2016'
column_to_use_as_prediction=[54,55]
column_to_predict='Goal/No_Goal'

In [108]:
mix=[allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict]

In [116]:
svm_linear = svm.SVC(kernel='linear')
a=Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_linear,'svm_linear')

In [117]:
b=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_linear,'svm_linear')

In [118]:
#c=Predict_season_with_current_season(allchampionships[0].df,season_to_be_predicted, column_to_use_as_prediction, column_to_predict,svm_linear,'svm_linear')

In [119]:
a['svm_linear'].mean()

0.601955782312925

In [121]:
b['svm_linear'].mean()

0.601955782312925

## SVM_RBF


In [110]:
svm_rbf = svm.SVC(kernel='rbf', gamma = 100)
a=Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict,svm_rbf,'svm_rbf')

In [111]:
b=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_rbf,'svm_rbf')

In [113]:
a['svm_rbf'].mean()

0.5593537414965987

In [114]:
b['svm_rbf'].mean()

0.5599489795918368

## CART

In [ ]:
dtc = DecisionTreeClassifier()
Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, dtc,'dtc')

In [ ]:
Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, dtc,'dtc')

# Adaboost Classifier 

In [ ]:
bagb = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10), n_estimators=20)
Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, bagb,'bagb')

In [ ]:
Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, bagb,'bagb')

# Gradient Boosting Classifier


In [ ]:
gradb = GradientBoostingClassifier(max_depth=10, n_estimators=100)
Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, gradb,'gradb')

In [ ]:
Predict_whole_season_with_one(Italy,'2015/2016',[46,47], 'FTR', gradb,'gradb')

# Random Forest

In [ ]:
rdf = RandomForestClassifier(max_depth=10, n_estimators=50)
Predict_whole_season_by_one(Italy,'2015/2016',[46,47], 'FTR', rdf,'rdf')

In [ ]:
Predict_whole_season_with_one(Italy,'2015/2016',[46,47], 'FTR', rdf,'rdf')

In [294]:
a=select_season(allchampionships[0].df, '2015/2016')

In [298]:
a.H_ROUND.unique()

array([  5.,   6.,   7.,   8.,   9.,  10.,  11.,  12.,  13.,  14.,  15.,
        16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.,  24.,  25.,  26.,
        27.,  28.,  29.,  30.,  31.,  32.])

In [356]:
g=30

In [357]:
Y_test=a[a.H_ROUND>g]['FTR']

In [358]:
X_test=a[a.H_ROUND>g].iloc[:,[50,51]]

In [359]:
Y_train=a[a.H_ROUND<=g]['FTR']
X_train=a[a.H_ROUND<=g].iloc[:,[50,51]]

In [360]:
svm_linear.fit(X_train,Y_train,)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [366]:
svm_linear.decision_function(X_test)

array([[ 0.69684217, -0.2480954 , -1.00002401],
       [ 0.15444954, -1.41276864, -0.99991077],
       [ 0.44449283, -0.33884951, -1.00003654],
       [ 0.21196258, -0.92246081, -0.9999758 ],
       [ 0.01261307, -0.94118426, -0.99999336],
       [ 0.24756597, -0.79192232, -0.99999106],
       [ 0.67845892,  0.4757038 , -1.00013051],
       [ 0.23072064, -0.48317437, -1.0000374 ],
       [ 0.78709045, -0.10184035, -1.00003598],
       [ 0.64821307, -0.13351353, -1.00004551]])

# Ridge & Lasso 

In [18]:
from sklearn.linear_model import Ridge, ridge_regression

In [21]:
allchampionships[3].df

,country_id,season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,H_ROUND,H_TP3,H_TP5,H_GS_H,H_GS_A,H_GR_H,H_GR_A,H_TGS,H_TGR,H_AvGS_H,H_AvGS_A,H_AvGR_H,H_AvGR_A,H_Av_TGS,H_Av_TGR,H_Pnt_H,H_Pnt_A,H_T_pnt,H_Av_Pnt_H,H_Av_pnt_A,H_Av_T_pnt,A_ROUND,A_TP3,A_TP5,Aw_GS_H,Aw_GS_A,Aw_GR_H,Aw_GR_A,Aw_TGS,Aw_TGR,Aw_AvGS_H,Aw_AvGS_A,Aw_AvGR_H,Aw_AvGR_A,Aw_Av_TGS,Aw_Av_TGR,Aw_Pnt_H,Aw_Pnt_A,Aw_T_pnt,Aw_Av_Pnt_H,Aw_Av_pnt_A,Aw_Av_T_pnt,DIFF_av_TP,DIFF_av_HP-AP,SUMM_av_HGSH+AGRA,SUMM_av_HGRH+AGSA,SUMM_av_HTGS+ATGR,SUMM_av_HTGR+ATGS,Goal/No_Goal,Over2.5/Under2.5
40,France Ligue 1,2008/2009,2008-09-13 00:00:00,Olympique Lyonnais,OGC Nice,3,2,H,5.0,7.0,3.0,5.0,1.0,0.0,0.0,6.0,0.0,2.500000,0.500000,0.000000,0.000000,1.500000,0.000000,6.0,4.0,10.0,3.000000,2.000000,2.500000,5.0,9.0,3.0,4.0,1.0,1.0,1.0,5.0,2.0,2.000000,0.500000,0.500000,0.500000,1.250000,0.500000,6.0,3.0,9.0,3.000000,1.500000,2.250000,0.250000,1.500000,3.000000,0.500000,2.000000,1.250000,1,1
41,France Ligue 1,2008/2009,2008-09-13 00:00:00,AS Monaco,FC Lorient,2,0,H,5.0,2.0,0.0,2.0,1.0,1.0,2.0,3.0,3.0,1.000000,0.500000,0.500000,1.000000,0.750000,0.750000,4.0,1.0,5.0,2.000000,0.500000,1.250000,5.0,1.0,0.0,0.0,2.0,2.0,3.0,2.0,5.0,0.000000,1.000000,1.000000,1.500000,0.500000,1.250000,1.0,3.0,4.0,0.500000,1.500000,1.000000,0.250000,0.500000,2.500000,1.500000,2.000000,1.250000,0,0
42,France Ligue 1,2008/2009,2008-09-13 00:00:00,Le Mans FC,Toulouse FC,1,2,A,5.0,9.0,3.0,1.0,7.0,1.0,2.0,8.0,3.0,0.500000,3.500000,0.500000,1.000000,2.000000,0.750000,3.0,6.0,9.0,1.500000,3.000000,2.250000,5.0,5.0,1.0,2.0,0.0,1.0,3.0,2.0,4.0,1.000000,0.000000,0.500000,1.500000,0.500000,1.000000,4.0,1.0,5.0,2.000000,0.500000,1.250000,1.000000,1.000000,2.000000,0.500000,3.000000,1.250000,1,1
43,France Ligue 1,2008/2009,2008-09-13 00:00:00,Stade Rennais FC,Le Havre AC,1,1,D,5.0,4.0,1.0,6.0,0.0,5.0,1.0,6.0,6.0,3.000000,0.000000,2.500000,0.500000,1.500000,1.500000,4.0,1.0,5.0,2.000000,0.500000,1.250000,5.0,0.0,0.0,1.0,2.0,1.0,4.0,3.0,5.0,0.500000,1.000000,0.500000,2.000000,0.750000,1.250000,3.0,0.0,3.0,1.500000,0.000000,0.750000,0.500000,2.000000,5.000000,3.500000,2.750000,2.250000,1,0
44,France Ligue 1,2008/2009,2008-09-13 00:00:00,Valenciennes FC,Grenoble Foot 38,1,1,D,5.0,3.0,0.0,4.0,1.0,1.0,5.0,5.0,6.0,2.000000,0.500000,0.500000,2.500000,1.250000,1.500000,6.0,0.0,6.0,3.000000,0.000000,1.500000,5.0,6.0,3.0,2.0,2.0,0.0,3.0,4.0,3.0,1.000000,1.000000,0.000000,1.500000,1.000000,0.750000,6.0,3.0,9.0,3.000000,1.500000,2.250000,-0.750000,1.500000,3.500000,1.500000,2.000000,2.500000,1,0
45,France Ligue 1,2008/2009,2008-09-13 00:00:00,Girondins de Bordeaux,Olympique de Marseille,1,1,D,5.0,3.0,0.0,4.0,1.0,1.0,3.0,5.0,4.0,2.000000,0.500000,0.500000,1.500000,1.250000,1.000000,6.0,0.0,6.0,3.000000,0.000000,1.500000,5.0,9.0,3.0,6.0,5.0,1.0,4.0,11.0,5.0,3.000000,2.500000,0.500000,2.000000,2.750000,1.250000,6.0,4.0,10.0,3.000000,2.000000,2.500000,-1.000000,1.000000,4.000000,3.000000,2.500000,3.750000,1,0
46,France Ligue 1,2008/2009,2008-09-13 00:00:00,AJ Auxerre,AS Nancy-Lorraine,1,1,D,5.0,3.0,3.0,2.0,2.0,2.0,4.0,4.0,6.0,1.000000,1.000000,1.000000,2.000000,1.000000,1.500000,3.0,3.0,6.0,1.500000,1.500000,1.500000,5.0,4.0,3.0,2.0,1.0,1.0,2.0,3.0,3.0,0.666667,1.000000,0.333333,2.000000,0.750000,0.750000,5.0,0.0,5.0,1.666667,0.000000,1.250000,0.250000,1.500000,3.000000,2.000000,1.750000,2.250000,1,0
47,France Ligue 1,2008/2009,2008-09-14 00:00:00,FC Sochaux-Montbéliard,LOSC Lille,1,1,D,5.0,1.0,0.0,2.0,2.0,3.0,4.0,4.0,7.0,1.000000,1.000000,1.500000,2.000000,1.000000,1.750000,1.0,0.0,1.0,0.500000,0.000000,0.250000,5.0,3.0,3.0,3.0,1.0,4.0,2.0,4.0,6.0,1.500000,0.500000,2.000000,1.000000,1.000000,1.500000,3.0,1.0,4.0,1.500000,0.500000,1.000000,-0.750000,0.000000,2.000000,2.000000,2.500000,2.750000,1,0
48,France Ligue 1,2008/2009,2008-09-14 00:00:00,SM Caen,AS Saint-Étienne,2,0,H,5.0,4.0,0.0,3.0,2.0,2.0,3.0,5.0,5.0,1.500000,1.000000,1.000000,1.500000,1.250000,1.250000,3.0,1.0,4.0,1.500000,0.500000,1.000000,5.0,3.0,0.0,2.0,0.0,2.0,2.0,2.0,4.0,1.000

In [35]:
y = allchampionships[3].df['Goal/No_Goal']
X = allchampionships[3].df.iloc[:,8:-2]

In [28]:
clf = Ridge(alpha=1.0)
clf.fit(X, y) 
Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [30]:
clf.get_params()

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': None,
 'normalize': False,
 'random_state': None,
 'solver': 'auto',
 'tol': 0.001}

In [31]:
import numpy as np
import matplotlib.pyplot as plt


class RidgeRegressor(object):
    """
    Linear Least Squares Regression with Tikhonov regularization.
    More simply called Ridge Regression.
    We wish to fit our model so both the least squares residuals and L2 norm
    of the parameters are minimized.
    argmin Theta ||X*Theta - y||^2 + alpha * ||Theta||^2
    A closed form solution is available.
    Theta = (X'X + G'G)^-1 X'y
    Where X contains the independent variables, y the dependent variable and G
    is matrix alpha * I, where alpha is called the regularization parameter.
    When alpha=0 the regression is equivalent to ordinary least squares.
    http://en.wikipedia.org/wiki/Linear_least_squares_(mathematics)
    http://en.wikipedia.org/wiki/Tikhonov_regularization
    http://en.wikipedia.org/wiki/Ordinary_least_squares
    """

    def fit(self, X, y, alpha=0):
        """
        Fits our model to our training data.
        Arguments
        ----------
        X: mxn matrix of m examples with n independent variables
        y: dependent variable vector for m examples
        alpha: regularization parameter. A value of 0 will model using the
        ordinary least squares regression.
        """
        X = np.hstack((np.ones((X.shape[0], 1)), X))
        G = alpha * np.eye(X.shape[1])
        G[0, 0] = 0  # Don't regularize bias
        self.params = np.dot(np.linalg.inv(np.dot(X.T, X) + np.dot(G.T, G)),
                             np.dot(X.T, y))

    def predict(self, X):
        """
        Predicts the dependent variable of new data using the model.
        The assumption here is that the new data is iid to the training data.
        Arguments
        ----------
        X: mxn matrix of m examples with n independent variables
        alpha: regularization parameter. Default of 0.
        Returns
        ----------
        Dependent variable vector for m examples
        """
        X = np.hstack((np.ones((X.shape[0], 1)), X))
        return np.dot(X, self.params)


if __name__ == '__main__':
    # Create synthetic data
    X = np.linspace(0, 6, 100)
    y = 1 + 2 * np.sin(X)
    yhat = y + .5 * np.random.normal(size=len(X))

    # Plot synthetic data
    plt.plot(X, y, 'g', label='y = 1 + 2 * sin(x)')
    plt.plot(X, yhat, 'rx', label='noisy samples')

    # Create feature matrix
    tX = np.array([X]).T
    tX = np.hstack((tX, np.power(tX, 2), np.power(tX, 3)))

    # Plot regressors
    r = RidgeRegressor()
    r.fit(tX, y)
    plt.plot(X, r.predict(tX), 'b', label=u'ŷ (alpha=0.0)')
    alpha = 3.0
    r.fit(tX, y, alpha)
    plt.plot(X, r.predict(tX), 'y', label=u'ŷ (alpha=%.1f)' % alpha)

    plt.legend()
    plt.show()

In [53]:
X.columns

Index([u'H_ROUND', u'H_TP3', u'H_TP5', u'H_GS_H', u'H_GS_A', u'H_GR_H',
       u'H_GR_A', u'H_TGS', u'H_TGR', u'H_AvGS_H', u'H_AvGS_A', u'H_AvGR_H',
       u'H_AvGR_A', u'H_Av_TGS', u'H_Av_TGR', u'H_Pnt_H', u'H_Pnt_A',
       u'H_T_pnt', u'H_Av_Pnt_H', u'H_Av_pnt_A', u'H_Av_T_pnt', u'A_ROUND',
       u'A_TP3', u'A_TP5', u'Aw_GS_H', u'Aw_GS_A', u'Aw_GR_H', u'Aw_GR_A',
       u'Aw_TGS', u'Aw_TGR', u'Aw_AvGS_H', u'Aw_AvGS_A', u'Aw_AvGR_H',
       u'Aw_AvGR_A', u'Aw_Av_TGS', u'Aw_Av_TGR', u'Aw_Pnt_H', u'Aw_Pnt_A',
       u'Aw_T_pnt', u'Aw_Av_Pnt_H', u'Aw_Av_pnt_A', u'Aw_Av_T_pnt',
       u'DIFF_av_TP', u'DIFF_av_HP-AP', u'SUMM_av_HGSH+AGRA',
       u'SUMM_av_HGRH+AGSA', u'SUMM_av_HTGS+ATGR', u'SUMM_av_HTGR+ATGS'],
      dtype='object')

In [61]:
x_col = ['rss', 'intercept']

In [62]:
for i in X.columns:
    x_col.append(i)

In [55]:
x_col + X.columns

C:\Users\Hrafnkell\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: using '+' to provide set union with Indexes is deprecated, use '|' or .union()
  if __name__ == '__main__':


ValueError: operands could not be broadcast together with shapes (2,) (48,) 

In [51]:
len(a)

50

In [37]:
from sklearn.linear_model import Ridge
def ridge_regression(independent, dependent, alpha):
    #Fit the model
    ridgereg = Ridge(alpha=alpha,normalize=True)
    ridgereg.fit(independent, dependent)
    y_pred = ridgereg.predict(independent)
    
    #Return the result in pre-defined format
    rss = sum((y_pred-y)**2)
    ret = [rss]
    ret.extend([ridgereg.intercept_])
    ret.extend(ridgereg.coef_)
    return ret

In [ ]:
#Initialize predictors to be set of 15 powers of x
predictors=['x']
predictors.extend(['x_%d'%i for i in range(2,16)])

#Set the different values of alpha to be tested
alpha_ridge = [1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20]

#Initialize the dataframe for storing coefficients.
col = ['rss','intercept'] + ['coef_x_%d'%i for i in range(1,16)]
ind = ['alpha_%.2g'%alpha_ridge[i] for i in range(0,10)]
coef_matrix_ridge = pd.DataFrame(index=ind, columns=col)

models_to_plot = {1e-15:231, 1e-10:232, 1e-4:233, 1e-3:234, 1e-2:235, 5:236}
for i in range(10):
    coef_matrix_ridge.iloc[i,] = ridge_regression(data, predictors, alpha_ridge[i], models_to_plot)

In [34]:
len(y)

100

In [77]:
a = ridge_regression(X, y, 2)

In [78]:
ridge_df = pd.DataFrame(columns=x_col)

In [79]:
ridge_df.loc[0] = a

In [80]:
pd.DataFrame(ridge_df.loc[0]).sort(0)

C:\Users\Hrafnkell\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,0
Aw_Av_Pnt_H,-0.013300
Aw_Av_T_pnt,-0.010193
Aw_AvGR_H,-0.009113
Aw_Av_TGR,-0.006166
H_Av_Pnt_H,-0.004780
H_Av_T_pnt,-0.003676
Aw_AvGS_H,-0.002584
DIFF_av_HP-AP,-0.002344
SUMM_av_HTGS+ATGR,-0.001236
A_TP3,-0.001141


In [47]:
a.sort()
a

[-0.056998738129122019,
 -0.050407975417085985,
 -0.043604104188755551,
 -0.03610477146095404,
 -0.035819070089922772,
 -0.020243028318298141,
 -0.017385359214918395,
 -0.013747069251568694,
 -0.012800814100744124,
 -0.008697944671740461,
 -0.00704150816908501,
 -0.006237741244048035,
 -0.0045337644480364734,
 -0.0032099734229380531,
 -0.0021671472454999289,
 -0.0019286395948959131,
 -0.0017696558801071722,
 -0.0015992359810678789,
 -0.0015410142574100625,
 -0.0015345179233338321,
 -0.0013959438519172117,
 -0.0013874004445176209,
 -0.00060702229448759699,
 -0.00050673867419669572,
 -0.00038158108698473927,
 -0.00018231138241160921,
 -0.00011720514228922822,
 3.4620043101386166e-05,
 0.00013745941032158702,
 0.0001929851198502901,
 0.00026494979890267974,
 0.00039220441494788199,
 0.00079132929607208852,
 0.0008521137409808659,
 0.0010966645868265589,
 0.0016096971957831338,
 0.0016348190185595461,
 0.0020368512822467508,
 0.0021878825474033721,
 0.0040431344153308291,
 0.00565861824421

In [39]:
a

[593.74806142067507,
 0.52378189739110392,
 -7.8845420205607297e-05,
 -0.0010300394018607148,
 -0.00083658215025725336,
 0.00056605315433380676,
 0.0010041439227673873,
 0.00028707121134424892,
 -0.00013330739401882694,
 0.00042675103375077817,
 1.2139234491834133e-05,
 0.0035543130755558176,
 0.0011342470268088242,
 0.010005162441482132,
 6.5964345862875239e-05,
 0.0040065043948628041,
 0.0011884981232584586,
 -3.446820060622001e-06,
 0.00044383252800297534,
 9.8896999100138094e-05,
 -0.006684304167954634,
 -0.0025590866796841717,
 -0.0056522500660938779,
 -7.007332639166278e-05,
 -0.0014869927236108912,
 -0.0013154645206815052,
 0.00016702797091006856,
 4.912554891077934e-06,
 -0.00058397455054640503,
 0.00014474148527495695,
 6.0771634104975367e-05,
 -7.6718755125388332e-05,
 -0.0012101680013943414,
 0.020514748249068144,
 -0.015956173424827167,
 -0.00080438410388517607,
 0.012981009034188409,
 -0.012143386725438439,
 -0.00063467822576402755,
 -0.00089379942128957844,
 -0.0004219147